In [3]:
import numpy as np
import cv2

def detect_blue(frame, background):
    # Convert image to HSV
    hsv_image = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)

    sensitivity = 20
    H_Value = 20    # Change this value if you want to segment some other color
    light_blue = np.array([H_Value - sensitivity, 60, 60])
    dark_blue = np.array([H_Value + sensitivity, 255, 255])
    mask = cv2.inRange(hsv_image, light_blue, dark_blue)    # Creating a segmentaion mask for blue color

    # Bigger the kernel size, lesser the gaps
    kernel_size = 10
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    closing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cont_sorted = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
    contour_mask = cv2.fillPoly(np.zeros((500, 500, 3), dtype=np.uint8), pts =[cont_sorted[0]], color=(255,255,255))

    object_mask = cv2.fillPoly(frame, pts =[cont_sorted[0]], color=(0,0,0))
    background_mask = np.bitwise_and(contour_mask, background)

    # Final image is created by doing a bitwise and of the two masks, which in turn removes the color in question
    # and replaces it with the background
    final_img = cv2.bitwise_or(object_mask, background_mask)
    
    return final_img



cap = cv2.VideoCapture(0)

ret, background = cap.read()
background = cv2.resize(background, (500, 500))
cv2.imshow('Background', background)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the video in MP4 format in the same directory as the code (Optional)
fourcc = cv2.VideoWriter_fourcc('MP4V')
out = cv2.VideoWriter('output.mp4', fourcc, 30.0, (500,500))

while(True):
    ret, frame = cap.read()
    frame = cv2.resize(frame, (500, 500))
    image = detect_blue(frame, background)

    out.write(image)

    cv2.imshow('Image',image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()